# Stepik ML Contest
Практическим проектом нашего [курса](https://stepik.org/course/4852/syllabus) будет анализ активности студентов онлайн курса "Введение в анализ данных в R".

Описание данных:
- [**events_train.csv**](https://stepik.org/media/attachments/course/4852/event_data_train.zip) - данные о действиях, которые совершают студенты со стэпами.
    1. step_id - id стэпа
    2. user_id - анонимизированный id юзера
    3. timestamp - время наступления события в формате unix date
    4. action - событие, возможные значения: 
        - discovered - пользователь перешел на стэп
        - viewed - просмотр шага,
        - started_attempt - начало попытки решить шаг, ранее нужно было явно нажать на кнопку - начать решение, перед тем как приступить к решению практического шага
        - passed - удачное решение практического шага. 
- [**submissions_train.csv**](https://stepik.org/media/attachments/course/4852/submissions_data_train.zip) - данные о времени и статусах сабмитов к практическим заданиям.
    1. step_id - id стэпа
    2. timestamp - время отправки решения в формате unix date
    3. submission_status - статус решения
    4. user_id - анонимизированный id юзера

### Мы будем считать, что пользователь успешно закончил курс, если он правильно решил больше 40 практических заданий.

В данных:
- [submission_data_test.csv](https://stepik.org/media/attachments/course/4852/submission_data_test.csv)
- [events_data_test.csv](https://stepik.org/media/attachments/course/4852/events_data_test.csv)

хранится информация о решениях и действиях для 6184 студентов за первые два дня прохождения курса. Это 6184 студентов, которые проходили курс в период с мая 2018 по январь 2019.

Используя данные о первых двух днях активности на курсе вам нужно предсказать, наберет ли пользователь более 40 баллов на курсе или нет.

В этих данных, вам доступны только первые дня активности студентов для того, чтобы сделать предсказание. На самом деле, используя эти данные, вы уже можете сделать прогноз. Например, если пользователь за первые два дня набрал 40 баллов, скорее всего он наберет более 40 баллов в дальнейшем. Чтобы подкрепить такие гипотезы, вы можете использовать данные, на которые мы исследовали в первых двух модулях курса, где для всех пользователей представлены все данные об их активности на курсе. 

## Соревнование начинается
<hr>

## 1. Построение baseline решения задачи.
Итак, у нас есть информация о том, какую активность проявляли пользователи за 2015-2018 года на курсе по "Введению в анализ данных на R" в виде датасета events_train. Также нам даны данные о попытках закрыть задачу в виде датафрейма submissions_train.
<br>Наша задача состоит в предсказании: пройдет ли студент курс на 40 баллов или же уйдет с курса. Предсказать его поведение нужно за первые два дня, проведенные на курсе.

В первую очередь определим, **что значит, что студент ушел с курса?** В принципе, мы можем взять какое-то определенное число, например 1 месяц, но было бы неплохо, если бы мы выяснили это каким-то вещим образом. Можем посчитать 90 и 95 персентиль и посчитать их среднее.

Теперь выясним, **какие данные могут помочь нам провести анализ за первые 2 дня и сделать предсказание?**:
<br>На самом деле, не так много. Как бы круто не была устроена модель, будем ориентироваться на адекватные возможности уместить данные за первые два дня.
- В качестве переменных для **y** мы возьмем значение переменной *passed_course* - прошел ли студент курс (на 40+ баллов).
- В **Х** мы можем сложить:
    1. **user_id** - идентификаторы пользователя, как индексы.
    2. **unique_days** - Количество уникальных проведенных на курсе дней (за первые дня дня). Максимум - это три дня (т.к. двое суток могут охватить 3 дня). Таким образом можно выяснить, насколько активен пользователь был в первые дни на курсе.
    3. Количество совершенных действий каждого типа action. Как и для предыдущего пункта. Пользователь мог отвалиться на каком то этапе.
    4. Количество правильно и неправильно решенных степов, а также процентное соотношение правильных попыток ко всем попыткам. Эти данные могут помочь словить "сгоревших" пользователей.
    5. **steps_tried** - В целом, количество шагов, которые пользователь попробовал решить.

В принципе, только такие данные мы и можем синтезировать из наших датасетов. Правда можно еще взять и учесть последний попробованный степ, для каждого пользователя. Может быть мы сумеем обнаружить такой степ, на котором много людей отсеиваются.

## 2. Предобработка данных
Ключевые аналитические моменты мы выделили, теперь проведем предобработку данных, проанализируем датасеты на адекватность, и посчитаем данные об успеваемости пользователей за все время. Также не забываем обращать внимание на сохранность наших данных, чтобы мы не потеряли никаких нужных наблюдений и не убрали ничего лишнего.

Импортируем библиотеки и загрузим все данные.

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

events_data_train      = pd.read_csv("datasets/event_data_train.zip", compression="zip")
submissions_data_train = pd.read_csv("datasets/submissions_data_train.zip", compression="zip")